# Preparação da base de dados para treinar os classificadores

In [1]:
from src import (
    Folder,
    GetModel,
    SegmentedList,
    segmentation_fails,
    ds2df,
    save_results,
    File2df,
    Best_rf,
    Rf,
    MLP,
    SVM,
    XGBoost,
    KNN,
    GBM,
    List2Best,
    Save_model
    )

from os.path import join
DATA = "Dados"
MODEL = "Models"

In [2]:
#adquindo as imagens e seus respectivos rotulos
#dataBasePath = join(DATA,"dados")
#dataBasePath = join(DATA,"changed","rotacionados")
dataBasePath = join(DATA,"changed","segmentation")
images, labels = Folder(dataBasePath)

## Caso as imagens não estejam segmentadas, podemos segmentalas

In [ ]:

segmentation_model = GetModel(join("models","best.pt"))
process_images = SegmentedList(images,segmentation_model,is_resized=False) # Por padrão redimensionar garante menor taxa de falha
#process_images = SegmentedList(images,segmentation_model,is_resized=True)

### com as imagens processadas, precisamos descartar as imagens que não foram segmentadas e salvar seu nome em um arquivo com as falhas

In [ ]:




fails = segmentation_fails(process_images,labels)

print(f"Quantidade: {len(fails)}\nFalhas:",fails)

#msg = "falhas-normais-"
#msg = "falhas-redimensionadas-"



### Podemos salvar as falhas em um arquivo de texto para vizualizar depois

In [ ]:
anotation_fails= join("docs","Falhas","teste.txt")

with open(anotation_fails, "w") as arquivo:
        for fail in fails:
            arquivo.write(fail + "\n")

### fazendo a filtragem das imagens válidas

In [ ]:
valid_images = []
valid_labels = []

for i in range(0,len(process_images),1):
    if labels[i] not in fails:
        valid_images.append(process_images[i])
        valid_labels.append(labels[i])
        

print(len(valid_labels),len(valid_images))

# Extraindo as Caracteristicas de interesse da base de dados

In [3]:
#df = ds2df(process_images,labels)
#df = ds2df(valid_images,valid_labels)
df = ds2df(images,labels)

### Visando evitar repetir o processamento para a mesma base de dados podemos exportar a mesma para um arquivo e apenas ler a partir do arquivo quando quisermos novamente

In [4]:

save_results(df,DATA,"segmentation-bruto",".csv")

### Carregando nossos dados a partir de um dataset .csv

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
df = File2df(join(DATA,"13-03-2024.csv"))

#df.drop("Unnamed: 0",axis=1,inplace=True)
df.columns

# Treinando os classificadores

In [ ]:
rf = Best_rf(df,join("docs","classification"))


### Vamos pegar a melhor versão de cada modelo

In [5]:
bests_list = []

funcs = [Rf, MLP,SVM,XGBoost,KNN,GBM]
names = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

args = (0.1,0.2,0.3)

for i,func in enumerate(funcs,0):

    for arg in args:
        
        _, accuracy, precision, recall, f1, kappa = func(df,arg)
        bests_list.append((names[i],arg,accuracy, precision, recall, f1, kappa))
    
bests_list

[('RF', 0.1, 96.55, 98.53, 95.71, 97.1, 92.85),
 ('RF', 0.2, 92.67, 91.18, 96.12, 93.58, 85.06),
 ('RF', 0.3, 89.08, 86.79, 94.85, 90.64, 77.6),
 ('MLP', 0.1, 70.69, 68.0, 97.14, 80.0, 31.05),
 ('MLP', 0.2, 70.69, 74.8, 71.32, 73.02, 40.98),
 ('MLP', 0.3, 72.13, 68.58, 92.27, 78.68, 40.85),
 ('SVM', 0.1, 76.72, 78.67, 84.29, 81.38, 50.44),
 ('SVM', 0.2, 74.14, 71.7, 88.37, 79.17, 46.03),
 ('SVM', 0.3, 68.39, 66.41, 87.63, 75.56, 33.16),
 ('XGBoost', 0.1, 98.28, 100.0, 97.14, 98.55, 96.42),
 ('XGBoost', 0.2, 94.83, 95.35, 95.35, 95.35, 89.52),
 ('XGBoost', 0.3, 94.25, 95.79, 93.81, 94.79, 88.38),
 ('KNN', 0.1, 92.24, 95.52, 91.43, 93.43, 83.97),
 ('KNN', 0.2, 85.34, 86.82, 86.82, 86.82, 70.32),
 ('KNN', 0.3, 83.91, 83.17, 89.18, 86.07, 67.08),
 ('GBM', 0.1, 93.1, 95.59, 92.86, 94.2, 85.7),
 ('GBM', 0.2, 87.93, 86.86, 92.25, 89.47, 75.36),
 ('GBM', 0.3, 87.07, 86.34, 91.24, 88.72, 73.6)]

In [6]:
output = join("docs","classification")

In [8]:
#best = List2Best(bests_list,output,f"{len(valid_images)}.csv",True)
best = List2Best(bests_list,output,f"{len(images)}.csv",True)
best

'XGBoost'

# caso gostemos do resultado e desejemos salvar o modelo

In [ ]:

Save_model(model,join(MODEL,'rf'))

# Avaliando o comparativo dos modelos 

In [ ]:
# caso não tenhamos nosso DataFrame carregado, podemos carregalo apartir de um arquivo
result_models = File2df(join(output,"not rf.csv"))

#result_models.drop("Unnamed: 0",axis=1,inplace=True)
result_models.columns

In [ ]:
result_models

In [ ]:
rotulos = result_models.Modelo.unique()
rotulos

In [ ]:
# Suponha que 'df' seja o DataFrame contendo os resultados dos modelos

# Rótulos dos modelos
model_labels = ["RF","MLP","SVM","XGBoost","KNN","GBM"]

# Lista para armazenar os melhores resultados de cada modelo
best_results = []

# Iterar sobre os rótulos dos modelos
for model_label in model_labels:
    # Filtrar o DataFrame para o modelo atual
    model_df = result_models[result_models['Modelo'] == model_label]
    
    # Encontrar a linha com o melhor resultado para o modelo atual
    best_row = model_df.loc[model_df['Total'].idxmax()]
    
    # Adicionar o resultado à lista de melhores resultados
    best_results.append(best_row)

import pandas as pd
# Criar um DataFrame com os melhores resultados
print(best_results)
best_results_df = pd.DataFrame(best_results)

# Exibir o DataFrame com os melhores resultados
print(best_results_df)
